In [1]:
import cv2
import glob
from scipy.spatial import distance
import math
import pickle
import numpy as np
import pandas as pd
from utility import *
from direction_change_distances import direction_change_distances
from sklearn.model_selection import train_test_split
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.metrics import accuracy_score
from matplotlib.font_manager import json_load
#import sys
#stdout = sys.stdout

In [2]:
# Read in csv file with video meta data
# each row needs to have following info: file path, video number, object, track
video_files = pd.read_csv('F:\\Sky360_videos\\video_files.csv')

In [23]:
"""Run this cell for each object and attach numbers to dist_list,direction_change_list_cleaned and y_in accordingly. These 
lists are concatenated in the next cell."""

Object = "Plant" # Plane or Bird or Plant
sequence_length = 10 # Sequence length to train on

"""For a given object get a list of distances [px] and direction changes [°] calculated between consecutive images. This
will produce a warning for instances when the magnitude of a direction vector is 0. Will be dealt with in the following
lines."""
dist_list3,direction_change_list = direction_change_distances(video_files,Object,sequence_length)

## Concatenation of input variables

"""direction_change_list contains NaNs here and there for the instances described above. Here it makes sense to just
replace nans with 0."""
direction_change_list_cleaned3 = clean_nans(direction_change_list)

# these are the object labels for model training
y_in3 = np.array([Object for _ in range(int(len(dist_list3)/sequence_length))], dtype='<U9')

# little test if input training data and labels have equal lengths
len(dist_list3)/sequence_length == len(y_in3)

c:\Users\Admin\Documents\UFO\Kinematics\Time_series_classification\utility.py:27: RuntimeWarning: invalid value encountered in double_scalars
  cos_theta = np.dot(a,b)/(magn_a * magn_b) # cos of angle between vector a and b
c:\Users\Admin\Documents\UFO\Kinematics\Time_series_classification\utility.py:28: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos(cos_theta) * 180/math.pi # angle in degree


True

In [24]:
"""concatenate model input data (dist_list, direction_change_list_cleaned) of different objects""" 

# input for model training
dist_list_all = list(dist_list) + list(dist_list2) + list(dist_list3)
direction_change_list_all = list(direction_change_list_cleaned) + list(direction_change_list_cleaned2) + list(direction_change_list_cleaned3)
# labels for model training
y = np.concatenate((y_in, y_in2,y_in3), axis=0)

In [7]:
print(len(y_in))
print(len(y_in2))
print(len(y_in3))

153
146
163


In [25]:
# Prepare input dataframe needed for the model
# First input variable: directional changes [px]
d = {}
n = 0
for i in range(0,len(dist_list_all)):
    if i % sequence_length == 0:
      d[n] = pd.Series(direction_change_list_all[i:i+sequence_length])
      n += 1
df = pd.Series(d).to_frame('Direction_changes')

# 2nd input variable: distances [°]
d = {}
n = 0
for i in range(0,len(dist_list_all)):
    if i % sequence_length == 0:
      d[n] = pd.Series(dist_list_all[i:i+sequence_length])
      n += 1
df2 = pd.Series(d).to_frame('Distances')

# Concatenate different input variables to one input dataframe
X = pd.concat([df, df2], axis=1)

In [26]:
# Little check: X.size divided by number of variables should equal y.size
print(X.size/2)
print(y.size)

1696.0
1696


In [28]:
# create datasets for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [29]:
# define the model
# check following link for the multitude of available algos: 
# https://www.sktime.org/en/stable/api_reference/classification.html

# here we just use the default values for now, has potential for improvements
classifier = HIVECOTEV2(
    stc_params={"n_shapelet_samples": 1000}, 
    drcif_params={"n_estimators": 25},
    arsenal_params={"n_estimators": 10},
    tde_params={"n_parameter_samples": 100},
    verbose=0,
)

In [30]:
# fit model
classifier.fit(X_train, y_train)

HIVECOTEV2(arsenal_params={'n_estimators': 10},
           drcif_params={'n_estimators': 25},
           stc_params={'n_shapelet_samples': 1000},
           tde_params={'n_parameter_samples': 100})

In [ ]:
# alternatively load model
# import trained classification model
with open('model.pkl', 'rb') as f:
    clf = pickle.load(f)

In [31]:
# test the trained model

#sys.stdout = stdout

y_pred = classifier.predict(X_test)
print(y_pred)
print(y_test)
y_pred_proba = classifier.predict_proba(X_test)
score = accuracy_score(y_test, y_pred)
print(score)

['Plane' 'Bird' 'Bird' 'Plane' 'Bird' 'Plane' 'Plant' 'Plane' 'Plane'
 'Plant' 'Bird' 'Plant' 'Plant' 'Plant' 'Plant' 'Plane' 'Plant' 'Plant'
 'Plane' 'Bird' 'Plane' 'Plant' 'Plant' 'Bird' 'Plant' 'Bird' 'Plant'
 'Plane' 'Plant' 'Plant' 'Bird' 'Plant' 'Plant' 'Bird' 'Bird' 'Plant'
 'Plant' 'Plant' 'Plant' 'Plane' 'Bird' 'Plant' 'Plane' 'Bird' 'Bird'
 'Plant' 'Plane' 'Bird' 'Bird' 'Plane' 'Bird' 'Plant' 'Bird' 'Bird'
 'Plant' 'Plane' 'Plant' 'Plant' 'Bird' 'Plant' 'Bird' 'Plane' 'Plant'
 'Bird' 'Bird' 'Plane' 'Bird' 'Plant' 'Bird' 'Bird' 'Plant' 'Plant'
 'Plane' 'Plane' 'Bird' 'Bird' 'Bird' 'Plant' 'Plane' 'Plane' 'Bird'
 'Bird' 'Plane' 'Plant' 'Plant' 'Plant' 'Plant' 'Bird' 'Plane' 'Plant'
 'Plane' 'Plane' 'Plane' 'Bird' 'Plant' 'Plant' 'Bird' 'Plant' 'Plane'
 'Plane' 'Bird' 'Bird' 'Plane' 'Plane' 'Plane' 'Bird' 'Plane' 'Bird'
 'Plane' 'Plant' 'Plane' 'Bird' 'Plane' 'Bird' 'Bird' 'Plant' 'Bird'
 'Plant' 'Plane' 'Plant' 'Plant' 'Plant' 'Plane' 'Plant' 'Bird' 'Plant'
 'Plane' 'Bird' 'Pla

['Bird' 'Plant' 'Plane' 'Plant' 'Plane' 'Plant' 'Bird' 'Bird' 'Plane'
 'Plane' 'Plane' 'Bird' 'Bird' 'Bird' 'Plane' 'Plane' 'Plane' 'Plant'
 'Plant' 'Plant' 'Plant' 'Plant' 'Plane' 'Plant' 'Plant' 'Plant' 'Plant'
 'Bird' 'Plant' 'Bird' 'Bird' 'Bird' 'Bird' 'Plane' 'Bird' 'Plant' 'Plane'
 'Bird' 'Bird' 'Plane' 'Bird' 'Bird' 'Bird' 'Plant' 'Plane' 'Bird' 'Bird'
 'Plane' 'Plane' 'Plane' 'Plant' 'Plant' 'Plane' 'Plant' 'Plant' 'Plant'
 'Plant' 'Bird' 'Plane' 'Plane' 'Plant' 'Plane' 'Plane' 'Bird' 'Bird'
 'Bird' 'Plane' 'Plane' 'Bird' 'Bird' 'Bird' 'Plane' 'Plane' 'Plane'
 'Bird' 'Plant' 'Bird' 'Plane' 'Plane' 'Bird' 'Plant' 'Plant' 'Plane'
 'Plane' 'Plant' 'Bird' 'Plane' 'Bird' 'Plane' 'Plane' 'Bird' 'Plant'
 'Bird' 'Plane' 'Plant' 'Bird' 'Plant' 'Plant' 'Plant' 'Plane' 'Plane'
 'Plant' 'Plant' 'Bird' 'Plant' 'Plane' 'Plant' 'Plant' 'Plant' 'Plane'
 'Plant' 'Plant' 'Plant' 'Plane' 'Bird' 'Bird']


In [33]:
# save model
with open('model_hivecotev2_bird_plant_plane_10.pkl','wb') as f:
    pickle.dump(classifier,f)